In [330]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Cargamos la matriz de estudiantes únicamente con datos de España (previamente filtrada)

In [331]:
# si tenéis que poner vuestro path aquí, comentad el de abajo y poned el vuestro :)
df_students_spain = pd.read_csv('C:\\Users\\saraf\\OneDrive\\Documentos\\Data Science and AI\\capstone project\\data\\df_students_spain.csv') # sara

In [332]:
df_students_spain.head()

,CNT,CNTRYID,CNTSCHID,CNTSTUID,CYC,NatCen,STRATUM,SUBNATIO,REGION,OECD,...,PV3MPRE,PV4MPRE,PV5MPRE,PV6MPRE,PV7MPRE,PV8MPRE,PV9MPRE,PV10MPRE,SENWT,VER_DAT
0,Spain,Spain,72400442.0,72400001.0,08MS,Spain,"ESP - stratum 15: Castilla - La Mancha,Public",Spain,Spain: Castile-La Mancha,Yes,...,507.720,482.708,473.829,518.818,510.025,474.003,486.331,513.218,0.32199,01MAY23:16:47:23
1,Spain,Spain,72400658.0,72400003.0,08MS,Spain,"ESP - stratum 31: País Vasco,Public,Public",Spain,Spain: Basque Country,Yes,...,474.867,522.442,480.818,416.860,535.585,458.814,419.433,525.250,0.09103,01MAY23:16:47:28
2,Spain,Spain,72400504.0,72400006.0,08MS,Spain,"ESP - stratum 01: Andalucía,Public",Spain,Spain: Andalusia,Yes,...,309.691,219.090,275.138,328.206,189.496,244.839,216.095,310.887,0.96973,01MAY23:16:47:19
3,Spain,Spain,72400761.0,72400007.0,08MS,Spain,"ESP - stratum 23: La Rioja,Public",Spain,Spain: La Rioja,Yes,...,355.727,403.009,412.793,331.181,364.536,440.398,419.701,291.878,0.05187,01MAY23:16:47:25
4,Spain,Spain,72400784.0,72400010.0,08MS,Spain,"ESP - stratum 26: Madrid,Private",Spain,Spain: Madrid,Yes,...,564.697,586.783,546.132,625.176,476.592,490.397,543.891,551.054,0.32544,01MAY23:16:47:26


In [333]:
print(list(df_students_spain.columns))

['CNT', 'CNTRYID', 'CNTSCHID', 'CNTSTUID', 'CYC', 'NatCen', 'STRATUM', 'SUBNATIO', 'REGION', 'OECD', 'ADMINMODE', 'LANGTEST_QQQ', 'LANGTEST_COG', 'LANGTEST_PAQ', 'Option_CT', 'Option_FL', 'Option_ICTQ', 'Option_WBQ', 'Option_PQ', 'Option_TQ', 'Option_UH', 'BOOKID', 'ST001D01T', 'ST003D02T', 'ST003D03T', 'ST004D01T', 'ST250Q01JA', 'ST250Q02JA', 'ST250Q03JA', 'ST250Q04JA', 'ST250Q05JA', 'ST250D06JA', 'ST250D07JA', 'ST251Q01JA', 'ST251Q02JA', 'ST251Q03JA', 'ST251Q04JA', 'ST251Q06JA', 'ST251Q07JA', 'ST251D08JA', 'ST251D09JA', 'ST253Q01JA', 'ST254Q01JA', 'ST254Q02JA', 'ST254Q03JA', 'ST254Q04JA', 'ST254Q05JA', 'ST254Q06JA', 'ST255Q01JA', 'ST256Q01JA', 'ST256Q02JA', 'ST256Q03JA', 'ST256Q06JA', 'ST256Q07JA', 'ST256Q08JA', 'ST256Q09JA', 'ST256Q10JA', 'ST230Q01JA', 'ST005Q01JA', 'ST006Q01JA', 'ST006Q02JA', 'ST006Q03JA', 'ST006Q04JA', 'ST006Q05JA', 'ST007Q01JA', 'ST008Q01JA', 'ST008Q02JA', 'ST008Q03JA', 'ST008Q04JA', 'ST008Q05JA', 'ST258Q01JA', 'ST259Q01JA', 'ST259Q02JA', 'ST019AQ01T', 'ST019BQ01

Empezamos descartando las columnas que sabemos que no vamos a usar. 

* Pesos estadísticos (W_FSTUWT*): No los vamos a usar para entrenar el modelo.

In [334]:
# elimiar columnas que empiezan por W_FSTURWT

df_students_spain = df_students_spain.drop(df_students_spain.filter(regex='^W_FSTURWT').columns, axis=1)


* Information and Communication (IC*): Relacionadas con el uso de recursos tecnológicos de los estudiantes, no son relevantes para nuestro modelo.

In [335]:
df_students_spain = df_students_spain.drop(df_students_spain.filter(regex='^IC').columns, axis=1)

* Financial Literacy (FL*): Relacionadas con la cultura financiera de los estudiantes, sus experiencias con la gestión del dinero y su comportamiento financiero. Tampoco vamos a incluirlas en el modelo.

In [336]:
df_students_spain = df_students_spain.drop(df_students_spain.filter(regex='^FL').columns, axis=1)

In [337]:
print(list(df_students_spain.columns))

['CNT', 'CNTRYID', 'CNTSCHID', 'CNTSTUID', 'CYC', 'NatCen', 'STRATUM', 'SUBNATIO', 'REGION', 'OECD', 'ADMINMODE', 'LANGTEST_QQQ', 'LANGTEST_COG', 'LANGTEST_PAQ', 'Option_CT', 'Option_FL', 'Option_ICTQ', 'Option_WBQ', 'Option_PQ', 'Option_TQ', 'Option_UH', 'BOOKID', 'ST001D01T', 'ST003D02T', 'ST003D03T', 'ST004D01T', 'ST250Q01JA', 'ST250Q02JA', 'ST250Q03JA', 'ST250Q04JA', 'ST250Q05JA', 'ST250D06JA', 'ST250D07JA', 'ST251Q01JA', 'ST251Q02JA', 'ST251Q03JA', 'ST251Q04JA', 'ST251Q06JA', 'ST251Q07JA', 'ST251D08JA', 'ST251D09JA', 'ST253Q01JA', 'ST254Q01JA', 'ST254Q02JA', 'ST254Q03JA', 'ST254Q04JA', 'ST254Q05JA', 'ST254Q06JA', 'ST255Q01JA', 'ST256Q01JA', 'ST256Q02JA', 'ST256Q03JA', 'ST256Q06JA', 'ST256Q07JA', 'ST256Q08JA', 'ST256Q09JA', 'ST256Q10JA', 'ST230Q01JA', 'ST005Q01JA', 'ST006Q01JA', 'ST006Q02JA', 'ST006Q03JA', 'ST006Q04JA', 'ST006Q05JA', 'ST007Q01JA', 'ST008Q01JA', 'ST008Q02JA', 'ST008Q03JA', 'ST008Q04JA', 'ST008Q05JA', 'ST258Q01JA', 'ST259Q01JA', 'ST259Q02JA', 'ST019AQ01T', 'ST019BQ01

In [338]:
# obtenemos todos los nombres de las columnas que empiezan por PV

for i in df_students_spain.columns[df_students_spain.columns.str.startswith('PV')]:
    print(i)

PV1MATH
PV2MATH
PV3MATH
PV4MATH
PV5MATH
PV6MATH
PV7MATH
PV8MATH
PV9MATH
PV10MATH
PV1READ
PV2READ
PV3READ
PV4READ
PV5READ
PV6READ
PV7READ
PV8READ
PV9READ
PV10READ
PV1SCIE
PV2SCIE
PV3SCIE
PV4SCIE
PV5SCIE
PV6SCIE
PV7SCIE
PV8SCIE
PV9SCIE
PV10SCIE
PV1MCCR
PV2MCCR
PV3MCCR
PV4MCCR
PV5MCCR
PV6MCCR
PV7MCCR
PV8MCCR
PV9MCCR
PV10MCCR
PV1MCQN
PV2MCQN
PV3MCQN
PV4MCQN
PV5MCQN
PV6MCQN
PV7MCQN
PV8MCQN
PV9MCQN
PV10MCQN
PV1MCSS
PV2MCSS
PV3MCSS
PV4MCSS
PV5MCSS
PV6MCSS
PV7MCSS
PV8MCSS
PV9MCSS
PV10MCSS
PV1MCUD
PV2MCUD
PV3MCUD
PV4MCUD
PV5MCUD
PV6MCUD
PV7MCUD
PV8MCUD
PV9MCUD
PV10MCUD
PV1MPEM
PV2MPEM
PV3MPEM
PV4MPEM
PV5MPEM
PV6MPEM
PV7MPEM
PV8MPEM
PV9MPEM
PV10MPEM
PV1MPFS
PV2MPFS
PV3MPFS
PV4MPFS
PV5MPFS
PV6MPFS
PV7MPFS
PV8MPFS
PV9MPFS
PV10MPFS
PV1MPIN
PV2MPIN
PV3MPIN
PV4MPIN
PV5MPIN
PV6MPIN
PV7MPIN
PV8MPIN
PV9MPIN
PV10MPIN
PV1MPRE
PV2MPRE
PV3MPRE
PV4MPRE
PV5MPRE
PV6MPRE
PV7MPRE
PV8MPRE
PV9MPRE
PV10MPRE


Para el resto de columnas más específicas, usaremos un compendio de códigos y su correspondiente significado, recogido en el archivo `compendio_df_students.xlsx`.

In [339]:
compendio = pd.read_excel('C:\\Users\\saraf\\OneDrive\\Documentos\\GitHub\\tfm_pisa_educacion\\compendio_df_students.xlsx')

In [340]:
compendio.drop('Unnamed: 0', axis=1, inplace=True)
compendio.head()

,Sigla,Explicacion
0,CNT,Country
1,CNTRYID,Country ID Country Identifier
2,CNTSCHID,Intl. School ID
3,CNTSTUID,Intl. Student ID
4,CYC,PISA Assessment Cycle (2 digits + 2 character ...


In [341]:
# definimos una función que reciba un código y devuelva su significado

def obtener_significado(codigo, df_compendio):
    resultado = df_compendio.loc[df_compendio['Sigla'] == codigo, 'Explicacion']
    return resultado.iloc[0] if not resultado.empty else "Not found"

In [342]:
obtener_significado('CNT', compendio)

'Country'

Procedemos a una selección más cuidadosa de las columnas que vamos a usar para nuestro modelo. En este caso, el objetivo es predecir el aprobado o suspenso de un alumno usando variables relacionadas con la implicación de padres y profesores.

In [343]:
# lista de variables seleccionadas:

peguntas_seleccionadas = ['ST267Q01JA', 'ST267Q02JA', 'ST267Q03JA', 'ST267Q04JA', 'ST267Q05JA', 'ST267Q06JA', 'ST267Q07JA', 'ST267Q07JA', 'ST034Q03TA', 'ST270Q01JA',
             'ST270Q02JA', 'ST270Q03JA', 'ST270Q04JA', 'ST297Q01JA', 'ST300Q01JA', 'ST300Q02JA', 'ST300Q03JA', 'ST300Q04JA', 'ST300Q05JA', 'ST300Q06JA',
             'ST300Q07JA', 'ST300Q08JA', 'ST300Q09JA', 'ST300Q10JA', 'ST348Q04JA', 'ST348Q08JA', 'ST353Q01JA', 'ST353Q02JA', 'ST353Q05JA', 'ST353Q06JA',
             'ST353Q07JA', 'ST353Q08JA', 'PA003Q01TA', 'PA003Q18WA', 'PA003Q05IA', 'PA003Q20WA', 'PA003Q12JA', 'PA008Q01TA', 'PA008Q02TA', 'PA008Q03TA',
             'PA008Q04TA', 'PA008Q05TA', 'PA008Q06NA', 'PA008Q07NA', 'PA008Q08NA', 'PA008Q09NA', 'PA008Q10NA', 'PA185Q01JA', 'PA041Q01TA', 'WB155Q08HA',
             'WB155Q09HA', 'WB163Q01HA', 'WB163Q02HA', 'WB163Q03HA', 'WB163Q04HA', 'WB163Q05HA', 'WB163Q06HA', 'WB163Q07HA', 'WB163Q08HA']

# faltan las de wellbeing

In [344]:
# comprobar que todas las variables están en el dataset

variables_no_encontradas = []
for i in peguntas_seleccionadas:
    if i not in df_students_spain.columns:
        variables_no_encontradas.append(i)
        print(i)

In [345]:
for column in df_students_spain.columns:
    print(column + ': ' + str(obtener_significado(column, compendio)))

CNT: Country
CNTRYID: Country ID Country Identifier
CNTSCHID: Intl. School ID
CNTSTUID: Intl. Student ID
CYC: PISA Assessment Cycle (2 digits + 2 character Assessment type - MS/FT)
NatCen: National Centre 6-digit Code
STRATUM: Stratum ID 5-character (cnt + original stratum ID)
SUBNATIO: Adjudicated sub-region code 7-digit code (3-digit country code + region ID + stratum ID)
REGION: REGION
OECD: OECD country
ADMINMODE: Mode of Respondent
LANGTEST_QQQ: Language of Questionnaire
LANGTEST_COG: Language of Assessment
LANGTEST_PAQ: Language of Parent Questionnaire
Option_CT: Creative Thinking Option
Option_FL: Financial Literacy Option
Option_ICTQ: ICT Questionnaire Option
Option_WBQ: Well-Being Questionnaire Option
Option_PQ: Parent Questionnaire Option
Option_TQ: Teacher Questionnaire Option
Option_UH: Une Heure Option
BOOKID: Form Identifier
ST001D01T: Student International Grade (Derived)
ST003D02T: Student (Standardized) Birth - Month
ST003D03T: Student (Standardized) Birth -Year
ST004D

In [346]:
df_students_clean = df_students_spain.copy()

Finalmente, a parte de las columnas correspondientes a las variables que vamos a emplear, deberemos conservar las columnas _Plausible Values_ del dataset, con las que vamos a definir el target `Aprobado`. Concretamente, determinaremos el Aprobado o No Aprobado usando los valores plausibles de las principales materias: Matemáticas, Ciencia y Lectura (PV1MATH a PV10MATH , PV1SCIE a PV10MATH  y PV1READ a PV10READ)

In [347]:
# tomar los valores de las columnas que empiezan por PV y terminan por MATH, SCIE O READ

columnas_target = [col for col in df_students_clean.columns if col.startswith('PV') and col.endswith(('MATH', 'SCIE', 'READ'))]
print(list(columnas_target))

['PV1MATH', 'PV2MATH', 'PV3MATH', 'PV4MATH', 'PV5MATH', 'PV6MATH', 'PV7MATH', 'PV8MATH', 'PV9MATH', 'PV10MATH', 'PV1READ', 'PV2READ', 'PV3READ', 'PV4READ', 'PV5READ', 'PV6READ', 'PV7READ', 'PV8READ', 'PV9READ', 'PV10READ', 'PV1SCIE', 'PV2SCIE', 'PV3SCIE', 'PV4SCIE', 'PV5SCIE', 'PV6SCIE', 'PV7SCIE', 'PV8SCIE', 'PV9SCIE', 'PV10SCIE']


In [348]:
# dataset resultante: (columnas presentes en preguntas_seleccionadas y columnas_target)

df_students_clean = df_students_clean[peguntas_seleccionadas + columnas_target]

In [349]:
df_students_clean.head()

,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q07JA,ST034Q03TA,ST270Q01JA,...,PV1SCIE,PV2SCIE,PV3SCIE,PV4SCIE,PV5SCIE,PV6SCIE,PV7SCIE,PV8SCIE,PV9SCIE,PV10SCIE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,506.912,520.943,445.915,447.578,510.355,469.425,485.229,520.293,509.465,520.461
1,Agree,Disagree,NaN,NaN,Agree,Strongly agree,Agree,Agree,Strongly agree,Some lessons,...,472.056,497.419,583.550,486.117,543.515,501.654,522.876,518.897,540.951,555.862
2,NaN,Agree,Agree,NaN,Agree,Agree,NaN,NaN,NaN,Some lessons,...,336.125,383.288,390.317,360.386,389.356,424.082,377.563,422.321,363.447,431.934
3,Agree,Disagree,Agree,Strongly disagree,NaN,NaN,Agree,Agree,Agree,Every lesson,...,413.822,324.202,343.227,284.083,330.103,351.840,353.562,381.369,366.970,350.518
4,NaN,Strongly disagree,Strongly disagree,NaN,Strongly disagree,Strongly disagree,NaN,NaN,Strongly agree,Most lessons,...,530.906,472.011,492.387,431.729,524.883,420.419,529.333,559.941,496.913,488.453


In [329]:
df_students_clean.columns

Index(['ST267Q01JA', 'ST267Q02JA', 'ST267Q03JA', 'ST267Q04JA', 'ST267Q05JA',
       'ST267Q06JA', 'ST267Q07JA', 'ST267Q07JA', 'ST034Q03TA', 'ST270Q01JA',
       'ST270Q02JA', 'ST270Q03JA', 'ST270Q04JA', 'ST297Q01JA', 'ST300Q01JA',
       'ST300Q02JA', 'ST300Q03JA', 'ST300Q04JA', 'ST300Q05JA', 'ST300Q06JA',
       'ST300Q07JA', 'ST300Q08JA', 'ST300Q09JA', 'ST300Q10JA', 'ST348Q04JA',
       'ST348Q08JA', 'ST353Q01JA', 'ST353Q02JA', 'ST353Q05JA', 'ST353Q06JA',
       'ST353Q07JA', 'ST353Q08JA', 'PA003Q01TA', 'PA003Q18WA', 'PA003Q05IA',
       'PA003Q20WA', 'PA003Q12JA', 'PA008Q01TA', 'PA008Q02TA', 'PA008Q03TA',
       'PA008Q04TA', 'PA008Q05TA', 'PA008Q06NA', 'PA008Q07NA', 'PA008Q08NA',
       'PA008Q09NA', 'PA008Q10NA', 'PA185Q01JA', 'PA041Q01TA', 'WB155Q08HA',
       'WB155Q09HA', 'WB163Q01HA', 'WB163Q02HA', 'WB163Q03HA', 'WB163Q04HA',
       'WB163Q05HA', 'WB163Q06HA', 'WB163Q07HA', 'WB163Q08HA', 'PV1MATH',
       'PV2MATH', 'PV3MATH', 'PV4MATH', 'PV5MATH', 'PV6MATH', 'PV7MATH',
      

In [350]:
df_students_clean.shape

(30800, 89)